# JavaScript in .NET Interactive Notebooks

A fundamental to remember is that the Jupyter notebook itself is built with HTML and JavaScript. .NET Interactive Notebooks [[GitHub](https://github.com/dotnet/interactive)] shows this quite readily:

In [1]:
#!javascript

console.log(`
    Is there a window object? [${typeof window === 'object'}]

    window.navigator.userAgent =
    ${window.navigator.userAgent}

    window.document.location.pathname = ${window.document.location.pathname}
    window.document.location.protocol = ${window.document.location.protocol}
`);


    Is there a window object? [true]

    window.navigator.userAgent =
    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Code/1.83.1 Chrome/114.0.5735.289 Electron/25.8.4 Safari/537.36

    window.document.location.pathname = /index.html
    window.document.location.protocol = vscode-webview:


We also see that `console.log` is needed to display output which is not as advanced as [the IPython magics](https://github.com/BryanWilhite/jupyter-central/blob/master/ipython/00-magic.ipynb) or the various renderers in [the Jupyter Typescript kernel](https://github.com/BryanWilhite/jupyter-central/blob/master/funkykb/typescript/README.ipynb).

## .NET Interactive JavaScript variables are _not_ shared between Jupyter cells


Another inconvenience is the lack of variable sharing:

In [2]:
#!javascript

const myConstant = 'Hello!';

In [3]:
#!javascript

console.log(myConstant);

Error: myConstant is not defined

Even the `#!share` magic I introduce in “[Variable sharing](https://github.com/BryanWilhite/jupyter-central/blob/master/dotnet-interactive/dotnet-interactive-02-variable-sharing.ipynb)” does not work:

In [4]:
#!javascript

#!share --from javascript myConstant

console.log(myConstant);

It looks like `#!share` lifted `myConstant` up into the Jupyter/Polyglot notebook variables. I can verify this by adding a new JavaScript `const`:

In [5]:
#!javascript

const myOtherConstant = 'Hello, again!';

## can the .NET Interactive JavaScript `interactive` object be used to share between cells?

Yes, the conventional `interactive` object [I introduce in another notebook](https://github.com/BryanWilhite/jupyter-central/blob/master/dotnet-interactive/dotnet-interactive-05-requirejs.ipynb) _can_ be used to share state across cells:

In [6]:
#!javascript

window.interactive.myProperty = 'Hello?';

In [7]:
#!javascript

console.log(interactive.myProperty);

Hello?

See “.NET Interactive JavaScript has jQuery!” below. 

## does .NET Interactive JavaScript interact with `#!html` magics?

In [8]:
#!html

<pre id="my-new-element">
    Hello world!
</pre>

Hello world!

In [9]:
#!javascript

const myNewElement = document.getElementById('my-new-element');

console.log(myNewElement.id);

Error: Cannot read properties of null (reading 'id')

The short answer is _yes_.

## does .NET Interactive JavaScript interact with `#!value` magics?

In [10]:
#!value --name someJson

{
    "what": "some JSON",
    "why": "to share it with another subkernel"
}

In [11]:
#!javascript

#!share someJson --from value

const o = JSON.parse(someJson);

console.log(o.why);

to share it with another subkernel

Yes, again!

## .NET Interactive JavaScript has jQuery!

In [another notebook](https://github.com/BryanWilhite/jupyter-central/blob/master/dotnet-interactive/dotnet-interactive-05-requirejs.ipynb), I list the members of the `window` object. This list clearly shows `$` and `jQuery`:

In [2]:
#!javascript

console.log(`Are \`$\` and \`jQuery\` the same? [${$ === jQuery && ($ !== undefined) && ($ !== null)}]`);

Are `$` and `jQuery` the same? [true]

Through some old-school “feature detection” we can infer the version of `jQuery`:

In [4]:
#!javascript

console.log(`
    Is this \`jQuery\` 3.7? [${typeof $.uniqueSort === 'function'}]
`);


    Is this `jQuery` 3.7? [true]


With `jQuery`, we can now be more professional about sharing data between notebook cells with `$.extend` [📖 [docs](https://api.jquery.com/jQuery.extend/#jQuery-extend-target-object1-objectN)]:

In [7]:
#!javascript

$.rx = {};

$.extend($.rx, {myProperty: 'Hello?'});

In [8]:
#!javascript

console.log($.rx.myProperty);

Hello?

Note that `jQuery` is not an idea from Microsoft. It is part of the home world of [Project Jupyter](https://jupyter.org/) (but, as of this writing, only mentioned once in passing [in the official documentation](https://docs.jupyter.org/en/latest/contributing/ipython-dev-guide/js_events.html#javascript-events)).

## can `jQuery` load custom CSS from a CDN?

Yes, custom CSS can be loaded inside a Jupyter notebook:

In [1]:
#!javascript

const linkId = 'font-awesome';
const location = 'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.2/css/all.min.css';

if($(`head>link#${linkId}`).length === 0) {
    $('<link>')
        .appendTo('head')
        .attr({
            id: linkId,
            type: 'text/css',
            rel: 'stylesheet',
            href: location
        });
}

And even a `#!html` magic can take advantage of the external file:

In [2]:
#!html

<i class="fa-regular fa-heart"></i>

But the longer answer includes caveats:

- CSS-framework files might make the notebook unreadable as it overrides defaults.
- The external file should not be recognized by default when the notebook is opened (unless the cell loading the file is run automatically).

## <!-- -->

🐙🐱[BryanWilhite](https://github.com/BryanWilhite)